# Plotting cities and connections

In [26]:
# Complete list of US state capitals and their latitudes and longitudes
cities = {
    "Montgomery": (32.3770, -86.3006),
    #"Juneau": (58.3019, -134.4197),
    "Phoenix": (33.4484, -112.0740),
    "Little Rock": (34.7465, -92.2896),
    "Sacramento": (38.5816, -121.4944),
    "Denver": (39.7392, -104.9903),
    "Hartford": (41.7658, -72.6734),
    "Dover": (39.1582, -75.5244),
    "Tallahassee": (30.4383, -84.2807),
    "Atlanta": (33.7490, -84.3880),
    #"Honolulu": (21.3070, -157.8584),
    "Boise": (43.6150, -116.2023),
    "Springfield": (39.7983, -89.6446),
    "Indianapolis": (39.7684, -86.1581),
    "Des Moines": (41.5868, -93.6250),
    "Topeka": (39.0473, -95.6752),
    "Frankfort": (38.2009, -84.8733),
    "Baton Rouge": (30.4571, -91.1875),
    "Augusta": (44.3106, -69.7795),
    "Annapolis": (38.9784, -76.4922),
    "Boston": (42.3601, -71.0589),
    "Lansing": (42.7335, -84.5555),
    "St. Paul": (44.9537, -93.0900),
    "Jackson": (32.2988, -90.1848),
    "Jefferson City": (38.5767, -92.1735),
    "Helena": (46.5891, -112.0391),
    "Lincoln": (40.8136, -96.7026),
    "Carson City": (39.1638, -119.7674),
    "Concord": (43.2081, -71.5376),
    "Trenton": (40.2206, -74.7597),
    "Santa Fe": (35.6860, -105.9378),
    "Albany": (42.6526, -73.7562),
    "Raleigh": (35.7804, -78.6391),
    "Bismarck": (46.8083, -100.7837),
    "Columbus": (39.9612, -82.9988),
    "Oklahoma City": (35.4720, -97.5201),
    "Salem": (44.9429, -123.0351),
    "Harrisburg": (40.2732, -76.8867),
    "Providence": (41.8240, -71.4128),
    "Columbia": (34.0007, -81.0348),
    "Pierre": (44.3683, -100.3510),
    "Nashville": (36.1627, -86.7816),
    "Austin": (30.2672, -97.7431),
    "Salt Lake City": (40.7608, -111.8910),
    "Montpelier": (44.2601, -72.5754),
    "Richmond": (37.5407, -77.4360),
    "Olympia": (47.0379, -122.9007),
    "Charleston": (38.3498, -81.6326),
    "Madison": (43.0731, -89.4012),
    "Cheyenne": (41.1400, -104.8202)
}

# Initialize the map centered around the US
m_cities = folium.Map(location=[37.0902, -95.7129], zoom_start=4)

# Add markers for each city
for city, coords in cities.items():
    folium.Marker([coords[0], coords[1]], tooltip=city).add_to(m_cities)

m_cities


In [27]:
# Function to compute distance between two lat-long points using the Haversine formula
def haversine(lat1, lon1, lat2, lon2):
    # Radius of the Earth in kilometers
    R = 6371.0
    
    # Convert degrees to radians
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    
    # Differences in coordinates
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    # Haversine formula
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    distance = R * c
    
    return distance

# Generate a distance matrix
n = len(cities)
distance_matrix = np.zeros((n, n))

for i, city1 in enumerate(cities):
    for j, city2 in enumerate(cities):
        distance_matrix[i, j] = haversine(cities[city1][0], cities[city1][1], cities[city2][0], cities[city2][1])

distance_matrix


array([[   0.        , 2402.70258243,  614.14652965, ...,  787.28813037,
        1219.96358139, 1910.14071116],
       [2402.70258243,    0.        , 1824.48493033, ..., 2782.41545441,
        2240.0516226 , 1068.32561258],
       [ 614.14652965, 1824.48493033,    0.        , ..., 1032.02899589,
         958.84012181, 1306.69203092],
       ...,
       [ 787.28813037, 2782.41545441, 1032.02899589, ...,    0.        ,
         838.76375709, 2000.5059997 ],
       [1219.96358139, 2240.0516226 ,  958.84012181, ...,  838.76375709,
           0.        , 1288.11796116],
       [1910.14071116, 1068.32561258, 1306.69203092, ..., 2000.5059997 ,
        1288.11796116,    0.        ]])

In [28]:
# Sample list of matchings (as city pairs)
matchings = [("Montgomery", "Phoenix"), 
             ("Little Rock", "Denver"), 
             ("Hartford", "Tallahassee"), 
             ("Dover", "Atlanta")]
import folium

# Initialize the map centered around the US
m = folium.Map(location=[37.0902, -95.7129], zoom_start=4)

# Add markers for each city
for city, coords in cities.items():
    folium.Marker([coords[0], coords[1]], tooltip=city).add_to(m)

# Add lines for matchings
for city1, city2 in matchings:
    folium.PolyLine([cities[city1], cities[city2]], color="red", weight=2.5, opacity=1).add_to(m)

m

# Some inital code

In [48]:
import gurobipy as gp
from itertools import combinations

# Recreate the model
m = gp.Model("CityMatching")

# Create variables
x = m.addVars(combinations(cities.keys(), 2),vtype=gp.GRB.BINARY, name="x")

# Create objective using the haversine formula to calculate distance
m.setObjective(gp.quicksum(haversine(cities[city1][0], cities[city1][1], 
                                     cities[city2][0], cities[city2][1]) * x[city1, city2] for city1, city2 in combinations(cities.keys(), 2)), 
               gp.GRB.MAXIMIZE) # possibly change to minimize

# Model is ready for further constraints and optimization
# .....


m.update()

m.write('CityMatching.lp')

m.optimize()


Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[x86])

CPU model: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 0 rows, 1128 columns and 0 nonzeros
Model fingerprint: 0xbb6b80ba
Variable types: 0 continuous, 1128 integer (1128 binary)
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [7e+01, 4e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective 1888229.6387

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 12 available processors)

Solution count 1: 1.88823e+06 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.888229638668e+06, best bound 1.888229638668e+06, gap 0.0000%


In [46]:
"""
    Plot the solution of the matching problem on a folium map.
    
    Args:
    - model (gurobipy.Model): Solved Gurobi model object.
    - variables (dict): Dictionary of Gurobi variables for city pairs.
    - cities (dict): Dictionary with city names as keys and (latitude, longitude) as values.
    
    Returns:
- folium.Map: Map object with plotted cities and their matchings.
"""
# Create a new folium map
solution_map = folium.Map(location=[37.0902, -95.7129], zoom_start=4)

# Add markers for each city
for city, coords in cities.items():
    folium.Marker([coords[0], coords[1]], tooltip=city).add_to(solution_map)

# Extract pairs from the solution and draw lines
for (city1, city2) in combinations(cities.keys(), 2):
    if x[city1,city2].x > 0.5:  # If x[i,j] is set to 1 in the solution
        #print(city1, city2)
        folium.PolyLine([cities[city1], cities[city2]], color="red", weight=2.5, opacity=1).add_to(solution_map)

solution_map